## load data

In [2]:
! pwd

/usr/src/app/notebooks


In [10]:
! python --version

Python 3.10.12


In [51]:
! python /usr/src/app/scripts/download_data.py

✅ downloaded skaters data from the 2018 regular season
✅ downloaded skaters data from the 2018 playoffs season
✅ downloaded skaters data from the 2019 regular season
✅ downloaded skaters data from the 2019 playoffs season
✅ downloaded skaters data from the 2020 regular season
✅ downloaded skaters data from the 2020 playoffs season
✅ downloaded skaters data from the 2021 regular season
✅ downloaded skaters data from the 2021 playoffs season
✅ downloaded skaters data from the 2022 playoffs season
✅ downloaded goalies data from the 2018 regular season
✅ downloaded goalies data from the 2018 playoffs season
✅ downloaded goalies data from the 2019 regular season
✅ downloaded goalies data from the 2019 playoffs season
✅ downloaded goalies data from the 2020 regular season
✅ downloaded goalies data from the 2020 playoffs season
✅ downloaded goalies data from the 2021 regular season
✅ downloaded goalies data from the 2021 playoffs season
✅ downloaded goalies data from the 2022 regular season
✅

---

In [18]:
from pathlib import Path

# analytics
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np

# plot
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [19]:
DATA_FOLDER = Path("../data")
DATA_FILEPATH = Path("../data/skaters/2022_regular.csv")

## shots data dictionary

In [20]:
# shots data dictionary

df_data_dict = (
    pd
    .read_csv(DATA_FOLDER / "data_dictionaries/shots.csv")
    .drop(["Unnamed: 2"], axis=1)
    .rename(columns={"Variable": "column_moneypuck", "Definition": "description"})
    .iloc[:124, :]
)
df_data_dict["column"] = df_data_dict.column_moneypuck
df_data_dict = df_data_dict.loc[:, sorted(df_data_dict.columns)]

df_data_dict.tail()
# df_data_dict.to_dict(orient="records")

,column,column_moneypuck,description
119,xShotWasOnGoal,xShotWasOnGoal,The probability the shot was on net. (Either a...
120,isHomeTeam,isHomeTeam,Set to 1 if the shooting team is the home team
121,shotWasOnGoal,shotWasOnGoal,Set to 1 if the shot was on net. (Either a goa...
122,teamCode,teamCode,The team code of the shooting team. For exampl...
123,arenaAdjustedXCordABS,arenaAdjustedXCordABS,Absolute value of the arenaAdjustedXCord


## players data dictionary

In [24]:
# players data dictionary

df_data_dict = (
    pd
    .read_csv(DATA_FOLDER / "data_dictionaries/players.csv")
    .rename(columns={
        "*****MoneyPuck.com Player and Team Data******": "column_moneypuck",
        "Unnamed: 1": "description",
    })
    .iloc[25:, :]
)
df_data_dict["column"] = df_data_dict.column_moneypuck
df_data_dict = df_data_dict.loc[:, sorted(df_data_dict.columns)]

df_data_dict.tail()
# df_data_dict.to_dict(orient="records")

,column,column_moneypuck,description
171,xGoalsAgainstAfterShifts,xGoalsAgainstAfterShifts,xGoals the opposing team gets between 1 and 5 ...
172,corsiForAfterShifts,corsiForAfterShifts,Shot attempts the player's team gets between 1...
173,corsiAgainstAfterShifts,corsiAgainstAfterShifts,Shot attempts the opposing team gets between 1...
174,fenwickForAfterShifts,fenwickForAfterShifts,Unblocked Shot attempts the player's team gets...
175,fenwickAgainstAfterShifts,fenwickAgainstAfterShifts,Unblocked Shot attempts the opposing team gets...


---

# TODO:

-- kod níže vložit do skriptů!

## insert data into PostgreSQL

In [1]:
import psycopg2 
import psycopg2.extras as extras 
import pandas as pd

from pathlib import Path

In [2]:
SEASON_PART_TO_ID = {
    "regular": "00", 
    "playoffs": "01",
}
TEAM_TO_ID = {
    "ANA": "00",
    "ARI": "01",
    "BOS": "02",
    "BUF": "03",
    "CAR": "04",
    "CBJ": "05",
    "CGY": "06",
    "CHI": "07",
    "COL": "08",
    "DAL": "09",
    "DET": "10",
    "EDM": "11",
    "FLA": "12",
    "LAK": "13",
    "MIN": "14",
    "MTL": "15",
    "NJD": "16",
    "NSH": "17",
    "NYI": "18",
    "NYR": "19",
    "OTT": "20",
    "PHI": "21",
    "PIT": "22",
    "SEA": "23",
    "SJS": "24",
    "STL": "25",
    "TBL": "26",
    "TOR": "27",
    "VAN": "28",
    "VGK": "29",
    "WPG": "30",
    "WSH": "31"
}
SITUATION_TO_ID = {
    "all": "00",
    "5on5": "01",
    "5on4": "02",
    "4on5": "03",
    "other": "04",
}

In [3]:
db_params = {
    "host": "host.docker.internal",
    "database": "nhl",
    "user": "user",
    "password": "password"
}

# connect to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# create a cursor
cur = conn.cursor()

In [4]:
def get_skaters_id(df: pd.DataFrame) -> pd.Series:
    return (
        df.season.astype(str)
        + df.season_part.map(SEASON_PART_TO_ID)
        + df.team.map(TEAM_TO_ID)
        + df.player_id.astype(str)
        + df.situation.map(SITUATION_TO_ID)
    ).astype(int)

def read_skaters_data(data_folder_path: Path) -> pd.DataFrame:
    def read_csv(data_path):
        df = pd.read_csv(data_path)
        
        # replace duplicate values in "team" column
        if "team" in df:
            df["team"] = df.team.replace({"N.J": "NJD", "T.B": "TBL", "S.J": "SJS", "L.A": "LAK"})
            
        # add new columns
        df.insert(loc=2, column="season_part", value=data_path.stem.split("_")[-1])
        df.insert(loc=0, column="id", value=get_skaters_id(df))

        return df
    
    return pd.concat(
        [read_csv(data_path) for data_path in data_folder_path.glob("*.csv")], 
        ignore_index=True,
    )


data_folder_path = Path("/usr/src/app/data/skaters/")
df_skaters = read_skaters_data(data_folder_path=data_folder_path)

In [8]:
# establish connection 
conn = psycopg2.connect(**db_params)

pandas_type_to_sql_type = {
    "int64": "int",
    "float64": "float",
    "object": "varchar",
}


def get_col_to_sql_type_updates(df: pd.DataFrame) -> dict:
    col_to_sql_type = {}
    
    # object columns
    for col in df.select_dtypes("object").columns:            
        col_len = df[col].str.len().max()
        
        if df[col].str.len().nunique() == 1:
            col_to_sql_type[col] = f"varchar ({col_len})"
            continue
            
        col_to_sql_type[col] = f"varchar ({int(col_len * 1.1)})"
        
    # id column
    col_to_sql_type["id"] = "bigint PRIMARY KEY"
    
    return col_to_sql_type


def create_table_from_pandas(
    table_name: str, 
    df: pd.DataFrame,
    conn: psycopg2.extensions.connection,
) -> None:
    # map pandas data type to sql data types
    col_to_sql_type = {
        **df.dtypes.astype("str").map(pandas_type_to_sql_type).to_dict(),
        **get_col_to_sql_type_updates(df=df)
    }
    
    # create sql query
    cols_and_types = ",\n".join(
        f"{col} {sql_type}" for col, sql_type in col_to_sql_type.items()
    )
    sql = f"""
        DROP TABLE IF EXISTS {table_name} CASCADE;
        CREATE TABLE IF NOT EXISTS {table_name}(\n{cols_and_types}\n);
    """
    
    # execute the query
    cursor = conn.cursor() 
    cursor.execute(sql)
    
    print(f"✅ created table `{table_name}`")


def insert_data_into_table(
    table_name: str,
    df: pd.DataFrame,
    conn: psycopg2.extensions.connection,
) -> None:  
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ",".join(list(df.columns)) 
  
    # create sql query
    sql = f"INSERT INTO {table_name}({cols}) VALUES %s ON CONFLICT DO NOTHING"
    cursor = conn.cursor()
    try: 
        extras.execute_values(cur=cursor, sql=sql, argslist=tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        conn.rollback() 
        cursor.close() 
        
        print(f"❌ failed to insert data into table `{table_name}`")
        print(f"error: {error}")
        return

    print(f"✅ data inserted into table `{table_name}`")
    cursor.close() 


create_table_from_pandas(
    table_name="skaters",
    df=df_skaters,
    conn=conn,
)
insert_data_into_table(
    table_name="skaters",
    df=df_skaters,
    conn=conn,
)

✅ created table `skaters`
✅ data inserted into table `skaters`


---

# download data from NHL official API

In [1]:
import itertools
import json

from pathlib import Path

# analytics
import pandas as pd

import requests

In [83]:
DATA_FOLDER = Path("/usr/src/app/data/")

SEASONS = (    
    "20152016",
    "20162017",
    "20172018",
    "20182019",
    "20192020",
    "20202021",
    "20212022",
    "20222023",
    "20232024",
)

SEASON_TYPES = ("regular", "playoffs")

URL_STANDINGS = "https://api-web.nhle.com/v1/standings/now"
URL_ROSTER = "https://api-web.nhle.com/v1/roster"
URL_GAMECENTER = "https://api-web.nhle.com/v1/gamecenter"

### teams

- because I haven't found any better resource yet, I get teams info from standings endpoint:
    - example: https://api-web.nhle.com/v1/standings/now

In [72]:
! python /usr/src/app/src/extract/teams.py

✅ Standings loaded successfully!
🎉 all data downloaded successfully


In [79]:
from src.extract.teams import read_team_abbrev_to_team_mapping

team_abbrev_to_team = read_team_abbrev_to_team_mapping()
team_abbrev_to_team

{'BOS': 'Boston Bruins',
 'BUF': 'Buffalo Sabres',
 'CAR': 'Carolina Hurricanes',
 'CBJ': 'Columbus Blue Jackets',
 'DET': 'Detroit Red Wings',
 'FLA': 'Florida Panthers',
 'MTL': 'Montréal Canadiens',
 'NJD': 'New Jersey Devils',
 'NYI': 'New York Islanders',
 'NYR': 'New York Rangers',
 'OTT': 'Ottawa Senators',
 'PHI': 'Philadelphia Flyers',
 'PIT': 'Pittsburgh Penguins',
 'TBL': 'Tampa Bay Lightning',
 'TOR': 'Toronto Maple Leafs',
 'WSH': 'Washington Capitals',
 'ANA': 'Anaheim Ducks',
 'ARI': 'Arizona Coyotes',
 'CGY': 'Calgary Flames',
 'CHI': 'Chicago Blackhawks',
 'COL': 'Colorado Avalanche',
 'DAL': 'Dallas Stars',
 'EDM': 'Edmonton Oilers',
 'LAK': 'Los Angeles Kings',
 'MIN': 'Minnesota Wild',
 'NSH': 'Nashville Predators',
 'SJS': 'San Jose Sharks',
 'SEA': 'Seattle Kraken',
 'STL': 'St. Louis Blues',
 'VAN': 'Vancouver Canucks',
 'VGK': 'Vegas Golden Knights',
 'WPG': 'Winnipeg Jets'}

### players

- basic info about players can be found in rosters
    - example: https://api-web.nhle.com/v1/roster/TOR/20232024
- more detailed info about players can be found in players landing, including carreer total, draft info, and current roster:
    - example: https://api-web.nhle.com/v1/player/8478402/landing

In [76]:
! python /usr/src/app/src/extract/players.py

✅ BOS roster loaded successfully!
✅ BUF roster loaded successfully!
✅ CAR roster loaded successfully!
✅ CBJ roster loaded successfully!
✅ DET roster loaded successfully!
✅ FLA roster loaded successfully!
✅ MTL roster loaded successfully!
✅ NJD roster loaded successfully!
✅ NYI roster loaded successfully!
✅ NYR roster loaded successfully!
✅ OTT roster loaded successfully!
✅ PHI roster loaded successfully!
✅ PIT roster loaded successfully!
✅ TBL roster loaded successfully!
✅ TOR roster loaded successfully!
✅ WSH roster loaded successfully!
✅ ANA roster loaded successfully!
✅ ARI roster loaded successfully!
✅ CGY roster loaded successfully!
✅ CHI roster loaded successfully!
✅ COL roster loaded successfully!
✅ DAL roster loaded successfully!
✅ EDM roster loaded successfully!
✅ LAK roster loaded successfully!
✅ MIN roster loaded successfully!
✅ NSH roster loaded successfully!
✅ SJS roster loaded successfully!
❌ Error: Roster for SEA 20152016 was NOT loaded!
❌ Error: Roster for SEA 20162017 

In [81]:
from src.extract.players import read_players_data

df_players = read_players_data()

In [88]:
# top 10 oldest players of current season

(
    df_players
    .loc[
        (df_players.last_active_season.astype(str) == SEASONS[-1])
    ]
    .sort_values(by="birth_date")
    .head(10)
)

,id,first_name,last_name,position,shoots_catches,height_cm,weight_kg,birth_date,birth_city,birth_country,birth_state_province,last_active_team,last_active_season,url_headshot
182,8470966,Mark,Giordano,D,L,183,92,1983-10-03,Toronto,CAN,ON,TOR,20232024,https://assets.nhle.com/mugs/nhl/20232024/TOR/...
170,8470794,Joe,Pavelski,C,R,185,89,1984-07-11,Plover,USA,WI,DAL,20232024,https://assets.nhle.com/mugs/nhl/20232024/DAL/...
130,8470594,Marc-Andre,Fleury,G,L,188,84,1984-11-28,Sorel,CAN,QC,MIN,20232024,https://assets.nhle.com/mugs/nhl/20232024/MIN/...
137,8470604,Jeff,Carter,C,R,191,99,1985-01-01,London,CAN,ON,PIT,20232024,https://assets.nhle.com/mugs/nhl/20232024/PIT/...
134,8470600,Ryan,Suter,D,L,185,91,1985-01-21,Madison,USA,WI,DAL,20232024,https://assets.nhle.com/mugs/nhl/20232024/DAL/...
1129,8477930,Pierre-Edouard,Bellemare,L,L,180,90,1985-03-06,Le Blanc-Mesnil,FRA,NaN,SEA,20232024,https://assets.nhle.com/mugs/nhl/20232024/SEA/...
144,8470613,Brent,Burns,D,R,196,104,1985-03-09,Barrie,CAN,ON,CAR,20232024,https://assets.nhle.com/mugs/nhl/20232024/CAR/...
149,8470621,Corey,Perry,R,R,191,94,1985-05-16,Peterborough,CAN,ON,CHI,20232024,https://assets.nhle.com/mugs/nhl/20232024/CHI/...
207,8471274,Alex,Goligoski,D,L,180,78,1985-07-30,Grand Rapids,USA,MN,MIN,20232024,https://assets.nhle.com/mugs/nhl/20232024/MIN/...
184,8471214,Alex,Ovechkin,L,R,191,108,1985-09-17,Moscow,RUS,NaN,WSH,20232024,https://assets.nhle.com/mugs/nhl/20232024/WSH/...


### download raw games

- example: https://api-web.nhle.com/v1/gamecenter/2023020260/play-by-play

In [7]:
! python /usr/src/app/src/extract/games.py

Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/opt/venv/lib/python3.10/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/venv/lib/python3.10/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/opt/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 492, in _make_request
    raise new_e
  File "/opt/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 468, in _make_request
    self._validate_conn(conn)
  File "/opt/venv/lib/python3.10/site-pac